In [3]:
import pandas as pd
statements = pd.read_csv('data/statements.csv', encoding='utf8')['statements'].values

In [4]:
statements[:5]

array(['Fashion week in NYC is in February ',
       'Roosevelt Field  site did Lindbergh begin his flight from in 1927',
       'Saudi Arabia  country is the holy city of Mecca located in',
       'the sun is  5778.0 hot',
       'banana  flavor filling did the original Twinkies have'],
      dtype=object)

In [7]:
import spacy
from nltk import Tree
en_nlp = spacy.load('en_core_web_lg')

In [22]:
def get_entities(text):
    nlp_doc = en_nlp(text)
    for token in nlp_doc:
        print('{0}: {1}, {2}({3})'.format(token,token.pos_,token.dep_,token.head))
    
    print('---------------------')
    for ent in nlp_doc.ents:
        print(ent.text, ent.label_)

# test spacy Entity Recogmition

In [36]:
# get NN, NNS
get_entities(statements[4])

banana: NOUN, compound(filling)
 : SPACE, (banana)
flavor: NOUN, compound(filling)
filling: VERB, nsubj(have)
did: VERB, advmod(filling)
the: DET, det(Twinkies)
original: ADJ, amod(Twinkies)
Twinkies: PROPN, nsubj(have)
have: VERB, ROOT(have)
---------------------
Twinkies ORG


# Test Wikification

In [38]:
import tagme
tagme.GCUBE_TOKEN = "c294fe63-88fa-40e1-b1d4-34b70c887b29-843339462"
for item in tagme.annotate(statements[2]).get_annotations(0.1):
    print(item)

Saudi Arabia -> Saudi Arabia (score: 0.5400270819664001)
country -> Nation state (score: 0.1376003473997116)
holy city -> Jerusalem (score: 0.16229085624217987)
Mecca -> Mecca (score: 0.6146639585494995)


# Understend what is the connection between entities

In [30]:
# get wikification of entities
# try find something in Wiki/DBPedia
# set statements

In [35]:
statements[2]

'Saudi Arabia  country is the holy city of Mecca located in'

In [ ]:
# semantic role labeling
allennlp
nlpnet
senna

In [39]:
from allennlp.commands import DEFAULT_MODELS
from allennlp.common.file_utils import cached_path
from allennlp.service.predictors import SemanticRoleLabelerPredictor
from allennlp.models.archival import load_archive

import spacy
from spacy.tokens import Token

class SRLComponent(object):
    '''
    A SpaCy pipeline component for SRL
    '''
    
    name = 'Semantic Role Labeler'

    def __init__(self):
        archive = load_archive(self._get_srl_model())
        self.predictor = SemanticRoleLabelerPredictor.from_archive(archive, "semantic-role-labeling")
        Token.set_extension('srl_arg0')
        Token.set_extension('srl_arg1')
        
    def __call__(self, doc):
        # See https://github.com/allenai/allennlp/blob/master/allennlp/service/predictors/semantic_role_labeler.py#L74
        words = [token.text for token in doc]
        for i, word in enumerate(doc):
            if word.pos_ == "VERB":
                verb = word.text
                verb_labels = [0 for _ in words]
                verb_labels[i] = 1
                instance = self.predictor._dataset_reader.text_to_instance(doc, verb_labels)
                output = self.predictor._model.forward_on_instance(instance, -1)
                tags = output['tags']

                # TODO: Tagging/dependencies can be done more elegant 
                if "B-ARG0" in tags:
                    start = tags.index("B-ARG0")
                    end = max([i for i, x in enumerate(tags) if x == "I-ARG0"] + [start]) + 1
                    word._.set("srl_arg0", doc[start:end])

                if "B-ARG1" in tags:
                    start = tags.index("B-ARG1")
                    end = max([i for i, x in enumerate(tags) if x == "I-ARG1"] + [start]) + 1
                    word._.set("srl_arg1", doc[start:end])
        
        return doc
    
    def _get_srl_model(self):
        return cached_path(DEFAULT_MODELS['semantic-role-labeling'])

    
def demo():
    nlp = spacy.load("en")
    nlp.add_pipe(SRLComponent(), after='ner')
    doc = nlp("Apple sold 1 million Plumbuses this month.")
    for w in doc:
        if w.pos_ == "VERB":
            print("('{}', '{}', '{}')".format(w._.srl_arg0, w, w._.srl_arg1)) 
            # ('Apple', 'sold', '1 million Plumbuses)

ModuleNotFoundError: No module named 'allennlp'

In [45]:
#senna
from practnlptools.tools import Annotator
annotator=Annotator()

annotator.getAnnotations("He created the robot and broke it after making it.")['srl']

****************************************************************************************************
default values:
senna path:
  
Dependencie parser:
 Z:\anaconda\envs\py_36\lib\site-packages\pntl\
Stanford parser clr java -cp stanford-parser.jar edu.stanford.nlp.trees.EnglishGrammaticalStructure -treeFile in.parse -collapsed
****************************************************************************************************


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: ''